This script needs to run in a computer with ImageJ installed with the plugin "Auto_Threshold" (latest verion tested 1.17.2)

Note : Relative path to the macro, only works if ImageJ isn't open, otherwise it uses the Macro folder from ImageJ as the current directory

In [60]:
import os
import sqlalchemy as sa
import pandas as pd
import cv2
import numpy as np
import re
from PIL import Image

function that translate the treatment given to the image after the threshold, depending on type

In [2]:
def returnType(treatment):
    type0 = "";
    
    type1 = """run("Close-");\n""";
    
    type2 = """run("Open");\n"""; 
    
    type3 = """run("Erode");\n"""
    
    type4 = """run("Dilate");\n"""
    
    type5 = type1+type2;
    
    type6 = type2+type1;
    
    types = [type0,type1,type2,type3,type4,type5,type6]
    return (types[treatment])

fuction that creates the macro that does the changes needed to all images from the given dataset and returns the path to the created macro:
    - src : specify the original dataset directory path
    - dest : specify the directory path to the new dataset
    - method : Threshold method 
    - treatment : pre-defined set of changes to make after the threshold algorithm
    - filename : substring of original images before the image number
    - changeback : switch the background with the foreground color before make the defined changes. (default = True)

In [3]:
def createMacro(src, dest,method,treatment,fileName,changeBack=True):
    try: 
        os.makedirs(dest)
    except OSError:
        if not os.path.isdir(dest):
            raise
    dirPath = dest+"\\"+method+str(treatment);
    os.mkdir(dirPath)
    
    files = os.listdir(src);
    frames = sum(1 for file in files if file.endswith(".tif"))
    
    source = src.replace("\\","/");
    dirPath +="/"
    dirPath = dirPath.replace("\\","/");
    dirPath = dirPath.replace("\0","\\0")
    macro = """for(i=0;i<{frames};i++){{
    if(i<10){{
    fileTif = "{fileName}00"+i+".tif";}}
    else if(i<100){{
    fileTif = "{fileName}0"+i+".tif";}}
    else{{
    fileTif = "{fileName}"+i+".tif";}}
    open("{source}"+fileTif);
    run("16-bit");""".format(frames=(frames),fileName=fileName,source=source,destination=dirPath)
    if method:
        macro+="""run("Auto Threshold...", "method={method}")""".format(method=method)

    if(changeBack):
        macro += """setOption("BlackBackground", true);
        run("Convert to Mask");
        """;
    macro += str(returnType(treatment));
    if(changeBack):
        macro += """setAutoThreshold("Default dark");
        """;
    macro += """saveAs("Tiff", "{destination}"+fileTif);
    close();
    }}
    """.format(frames=frames,fileName=fileName,source=source,destination=dirPath,method=method)
    macroPath = dirPath+"macro.ijm"
    f = open(macroPath,'w');
    f.write(macro);
    f.close();
    return macroPath;
    

In [ ]:
def main():
    src = "..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\01_GT\\TRA\\segmented\\0\\";
    dest = "..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\01_GT\\TRA\\segmented\\0\\0\\";
    method = ""
    treatment = 0;
    fileName = "mask"
    macroPath = createMacro(src,dest,method,treatment,fileName,False)
    imageJdir = "C:\Users\cool_\Documents\ImageJ\ImageJ.exe";
    
    cmd = 'cmd /c \""'+ imageJdir + '" -macro \"' + macroPath + '\""';
    os.system(cmd);

main()

rename all images name to a format like name + number with 3 digits + ".tif"

In [ ]:
def renameImages(src,name):
    i=0;
    for file in os.listdir(src):
        os.rename(src+file,src+name+"{:03.0f}.tif".format(i));
        i +=1;

src = "..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\01_GT\\TRA\\segmented\\0\\0\\0\\";
renameImages(src,"mask")

In [23]:
def initializeDb(username, password, server, port, database):
    parameters = { 
               'username': username, 
               'password': password,
               'server':   server,
                'port' : port,
               'database': database
             }
    connection = 'postgresql://{username}:{password}@{server}:{port}/{database}'.format(**parameters);
    
    return sa.create_engine(connection, encoding="utf-8")

In [6]:
def main2 (username,password,server,port,database):
   
    return initializeDb(username,password,server,port,database);

In [66]:
engine = main2("postgres","postgres","localhost",5434,"Fluo-N2DH-SIM+_GT")

In [67]:
cells0 = pd.read_sql_query('select cellid, ST_AsText(ST_centroid(geom)) as centroid, myId from %s where frame=0'%('lifetmp'),con=engine)

In [ ]:
278 481 <type 'int'> <type 'int'>
278 481 <type 'int'> <type 'int'>

In [61]:
def changePix(image):
    for k in range(0,64):
        im = Image.open('..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\01_GT\\TRA\\segmented\\0\\0\\0\\man_track{:03.0f}.tif'.format(k)) # Can be many different formats.
        pix = im.load()
        cells = pd.read_sql_query('select cellid, ST_X(ST_centroid(geom)) as x,ST_Y(ST_centroid(geom)) as y, myId from %s where frame=%s'%('lifetmp',k),con=engine)
        for index, row in cells.iterrows():
            x = int(row.x)
            y = 0-int(row.y)
            print x,y,type(x),type(y)
            changeCell = pix[x,y]  # Get the RGBA Value of the apixel of an image
            columnsize, rowsize = im.size 
            for i in range(rowsize):
                for j in range(columnsize):
                    if(pix[j,i]==changeCell):
                        im.putpixel((j, i), row.myid)  # Set the RGBA Value of the image (tuple)
        
        im.save('RES\\mask{:03.0f}.tif'.format(k))  # Save the modified pixels as .png

In [119]:
def changePix(image):
    for k in range(0,1):
        im = Image.open('..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\01_GT\\TRA\\segmented\\0\\0\\0\\man_track{:03.0f}.tif'.format(k)) # Can be many different formats.
        pix = im.load()
        cells = pd.read_sql_query('select cellid, ST_AsText(ST_centroid(geom)) as centroid, myId + 256 myid from %s where frame=%s'%('lifetmp',k),con=engine)
        print cells
        for index, row in cells.iterrows():
            if row.myid == 0:
                continue
            try:
                print row.myid
                x = int(float(re.findall("\d+\.\d+",row.centroid)[0]))
                y = int(float(re.findall("\d+\.\d+",row.centroid)[1]))
                changeCell = pix[x,y]  # Get the RGBA Value of the apixel of an image
                columnsize, rowsize = im.size 
                for i in range(rowsize):
                    for j in range(columnsize):
                        if(pix[j,i]==changeCell):
                            im.putpixel((j, i), int(row.myid) )  # Set the RGBA Value of the image (tuple)
            except:
                print row.cellid
                print re.findall("\d+\.\d+",row.centroid)
        im.save('RES\\mask{:03.0f}.tif'.format(k))  # Save the modified pixels as .png

In [120]:
changePix('..\\mask000.tif')

    cellid                                   centroid  myid
0       12  POINT(172.485653235653 -445.570436507937)   269
1       19  POINT(615.218705547653 -216.883475580844)   276
2       28  POINT(512.394959349593 -79.8433333333333)   285
3       25  POINT(54.9831321899922 -147.097536876279)   282
4       13  POINT(409.805397826849 -423.220609884332)   270
5       26  POINT(310.185142769889 -107.311192548481)   283
6       14  POINT(105.294299735749 -418.474216685542)   271
7        3  POINT(318.001547867812 -629.627583004411)   260
8       15  POINT(604.815607531627 -420.122609591056)   272
9       16  POINT(49.0873786407767 -402.952877101587)   273
10      17  POINT(241.978603603604 -398.950530888031)   274
11      18  POINT(39.9768669124557 -345.170172775248)   275
12       2  POINT(600.959163095538 -637.265103772792)   259
13      11  POINT(52.7245349867139 -457.702096250369)   268
14      22  POINT(310.033053515215 -192.249300454704)   279
15      20   POINT(92.182841068917 -211.